In [1]:
#Import useful libraries
import pandas as pd
import datetime
import matplotlib.pyplot as plt

In [3]:
#load dataset used for analysis
dtypes = {"id":'str',"timestamp":'str',"longitude":'float',"latitude":'float',"annotation":'str',"speed":'float',"heading":'str',"shiptype":'str'}
data_or = pd.read_csv("ais_brest_synopses.csv",usecols=["id","timestamp","longitude","latitude","annotation","speed","heading"],dtype=dtypes)
dtypes_ships = {"id":'str',"shiptype":'str'}
data_ships = pd.read_csv("ship_list.csv",usecols=["id","shiptype"],dtype=dtypes_ships,sep=';')

In [ ]:
#Aggiunta Prorietà FISHING ad un messaggio di una nave da pesca
data_out = pd.merge(data,data_ships,on='id',how='left').fillna("none")

In [9]:
#Export csv
data.to_csv("ais_data_type.csv",sep=',',index=False)

In [ ]:
#Counting Base Annotation
annotation_simple_dict = {
    "10000000": 0,
    "01000000": 0,
    "00100000": 0,
    "00010000": 0,
    "00001000": 0,
    "00000100": 0,
    "00000010": 0,
    "00000001": 0
}
#Count type of Annotation
most_frequent=data["annotation"].value_counts()

#Convert to dict
most_dict=most_frequent.to_dict()
list_value = list(most_dict.values())
#Count single annotation
j = 0
for key in most_dict.keys():
    i = 0
    for element in key:
        if element == '1':
            list_key = list(annotation_simple_dict.keys())
            annotation_simple_dict[list_key[i]] += list_value[j]
        i += 1
    j += 1  
annotation_simple_dict

In [ ]:
#ANNOTATION BASE
#Counting Annotation base
plt.bar(x=annotation_simple_dict.keys(),height=list(annotation_simple_dict.values()))
plt.xticks(rotation=90)
plt.show()

In [ ]:
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
data['month'] = pd.DatetimeIndex(data['timestamp']).month
data['year'] = pd.DatetimeIndex(data['timestamp']).year
data.head(5)

In [ ]:
result=data[data['id']=="228133000"]
result

In [ ]:
#COUNT EVENT for each Month
#month_analysis= data[['annotation', 'month','year']].groupby(['month'])['annotation'].count().reset_index(name='count_annotation')
month_analysis= data[['annotation', 'month','year']].groupby(['month','year']).agg('count')
month_analysis
month_analysis.plot(xlabel='month,year',kind='bar',width=0.8)

In [ ]:
#COUNT EACH ANNOTATION FOR EACH MONTH
#month_analysis_annotation= data[['annotation','month']].groupby(['month']).agg(['annotation']).reset_index(name='count')
month_analysis_annotation=data.groupby(['month','annotation'])['annotation'].count().reset_index(name='count')
month_analysis_annotation
for i in range(1,13):
    name="month "+str(i)
    plt.figure()
    plt.title(name)
    df_monthly = month_analysis_annotation[month_analysis_annotation['month'] == i] # select dataframe with month = i
    plt.bar(df_monthly['annotation'], df_monthly['count'])
    plt.xticks(df_monthly['annotation'].unique(),rotation=90) # set x axis 
#month_analysis_annotation.plot.bar(xlabel=['annotation'],subplots=True,)

In [ ]:
#GAP ANALYSIS ANNOTATION
#month_analysis_annotation= data[['annotation','month']].groupby(['month']).agg(['annotation']).reset_index(name='count')
gap_analysis =data.query('annotation == "00010000"')
gap_analysis_annotation=gap_analysis.groupby(['month','id'])['annotation'].count().reset_index(name='count')
gap_analysis_annotation= gap_analysis_annotation.query('count >= 50')

for i in range(1,13):
    name="month "+str(i)
    plt.figure()
    plt.title(name)
    df_monthly = gap_analysis_annotation[gap_analysis_annotation['month'] == i] # select dataframe with month = i
    plt.bar(df_monthly['id'], df_monthly['count'])
    plt.xticks(df_monthly['id'].unique(),rotation=90) # set x axis 
#month_analysis_annotation.plot.bar(xlabel=['annotation'],subplots=True,)

In [ ]:
#EVALUATE OF maritime traffic in a certain area evaluated for every month included in dataset

traffic= data[['month','year','longitude','latitude']].query('longitude >= -4.46 and longitude <=-4.44 and latitude >= 48 and latitude <=48.15').groupby(['month','year'])['month'].agg('count')
traffic
traffic.plot(x='month,year',kind='bar',width=0.8)

In [ ]:
#EVALUATE OF maritime traffic in a certain area evaluated for every month included in dataset
#gap_analysis =data[data['annotation'].str.contains("^(?:001|011|101|111)")]
heading_change = data[data.annotation.str.startswith(('001','011','101','111'), na=False)]
#heading_change = data[data.annotation.str.startswith(('001'), na=False)]
Trawling_movement= heading_change.query('longitude >= -4.879198 and longitude <=-4.545699 and latitude >= 48.044979 and latitude <=48.335456')

In [ ]:
Trawling_movement.head(100)
Trawling_result=Trawling_movement[Trawling_movement['id']=="228854000"]
#Trawling_result[Trawling_result['timestamp']=="1443655028000"]


In [ ]:
Trawling_total = Trawling_movement.query('speed>=1.00 and speed <=9.00')

In [ ]:
Trawling_count =Trawling_total.groupby(data['id']).size() 

In [ ]:
Trawling_count.head(100)
